## How to fine-tune a LLM

### Fine-tuning LLM
* Purpose: Fine-tuning adjusts a pre-existing LLM to perform better on a specific task or domain, utilizing a smaller, domain-specific dataset.
* Advantages: Improves model performance for context-specific results and reduces training costs.

### Steps in Fine-tuning
1. Select a  Pre-trained Model: Choose a base model aligned with the desired functionalities. (e.g. Microsoft's Phi-2, Meta's LLaMA)
2. Gather Relevant Dataset: Collect a relevant dataset, with necessary labels or structure. This part is many times considered the real challenge.
3. Preprocess Dataset: Clean the data and split it into training, validation, and test sets.
4. Fine-tuning: Adjust the model on the prepared dataset to specialize for the specific task.
5. Further tailor the model in such a way that it becomes more attuned to the specific requirements, language, and knowledge of the task or domain it will be used for.

### Fine-tuning methods

* Full Fine-tuning: Updates all model weights to enhance performance across various tasks.
* Parameter Efficient Fine-Tuning (PEFT): Updates only a subset of parameters, reducing memory requirements and preserving learned information.

### Popular Fine-tuning methods: LoRA and QLoRA 

#### LoRA (Low-Rank Adaptation)

Here's how it works:
* Weight Matrix Approximation: Instead of updating the entire weight matrix of the pre-trained model (which can be very very large and resource-intensive), LoRA focuses on modifying smaller matrices. These smaller matrices are designed to approximate changes to the larger weight matrix effectively. 
* LoRA Adapters: The smaller matrices, known as "LoRA adapters," are inserted into the model. These adapters capture the necessary adjustments to the model's weights to adapt it to the specific task or domain.
* Efficiency and Performance: By fine-tuning only these smaller matrices, you can achieve similar performance improvements as full model fine-tuning but with significantly less computational cost. 

Here's a simple example that can be helpful to understand LoRA better:

Imagine you have a 6 x 6 large weight matrix W, which is the entire weight matrix of the pre-trained model. (In real LLMs, W has billions of rows and columns).

In [8]:
import numpy as np

W = np.random.randint(1, 10, size=(6, 6))
print(f"W: \n{W}")

W: 
[[2 6 4 2 3 8]
 [4 5 4 3 6 8]
 [9 2 1 5 9 1]
 [2 2 9 6 5 5]
 [1 4 9 1 5 6]
 [7 3 8 5 4 1]]


In LoRA, instead of fine-tuning all the elements of W, we approximate changes to W using two smaller matrices, A and B. Let's say A is 6x2 matrix and B is a 2x6 matrix:

In [5]:
A = np.random.random((6,2))
B = np.random.random((2,6))
print(f"A: \n{A}")
print(f"B: \n{B}")

A: 
[[0.67149334 0.43698063]
 [0.76943609 0.13184095]
 [0.74918086 0.69805827]
 [0.74757621 0.01147748]
 [0.14243444 0.28975049]
 [0.17845079 0.60621866]]
B: 
[[0.52789592 0.62018729 0.07316205 0.6132555  0.48515602 0.10829241]
 [0.86703269 0.96251119 0.59437175 0.50047613 0.52704569 0.64197135]]


Finding the matrices A and B in LoRA is part of the fine-tuning process. Initially, A and B are randomly initialized and then updated through training to better approximate the changes needed in the large weight matrix W for the specific task.

The product of A and B, which we'll call C, is a 6x6 matrix.

In [7]:
C = np.dot(A, B)
print(f"C: \n{C}")

C: 
[[0.73335508 0.83705038 0.30885677 0.63049536 0.55608779 0.35324668]
 [0.52049258 0.60409287 0.13465606 0.53784417 0.44278276 0.1679622 ]
 [1.00072885 1.13652134 0.46971773 0.80880078 0.7313782  0.52926401]
 [0.40459378 0.47468447 0.0615161  0.46419943 0.36874026 0.08832505]
 [0.3264137  0.36722411 0.1826403  0.2323619  0.22181467 0.20143608]
 [0.61981483 0.69416515 0.37337507 0.4128339  0.40608141 0.40849987]]


In LoRA, the original large matrix W is not directly modified during fine-tuning. Instead, the smaller matrices A and B are find-tuned, and their product C is used to adjust W during the model's forward pass. 

So, the model uses an effectice weight matrix W' which is the sume of the original W and the low-rank approximation C:

$$
W' = W + C
$$



In [9]:
W_prime = W + C
print(f"W': \n{W_prime}")

W': 
[[ 2.73335508  6.83705038  4.30885677  2.63049536  3.55608779  8.35324668]
 [ 4.52049258  5.60409287  4.13465606  3.53784417  6.44278276  8.1679622 ]
 [10.00072885  3.13652134  1.46971773  5.80880078  9.7313782   1.52926401]
 [ 2.40459378  2.47468447  9.0615161   6.46419943  5.36874026  5.08832505]
 [ 1.3264137   4.36722411  9.1826403   1.2323619   5.22181467  6.20143608]
 [ 7.61981483  3.69416515  8.37337507  5.4128339   4.40608141  1.40849987]]



#### QLoRA (Quantized Low-Rank Adaptation)

* Method: In addition to using low-rank matrices like LoRA, QLoRA quantizes these matrices, meaning the elements of A and B are represented with fewer bits. For example, instead of using 32-bit floating-point numbers, it might use 8-bit or 4-bit representations.
    * Image we have a set of 32-bit floating-point numbers: [0.25, 0.5, 0.75, 1.0]; These numbers can have a large range of values with many decimal places. 
    * For simplicity, let's assume these numbers are already scaled between 0 and 1 and we want to map them to 8-bit unsigned integers (which can represent values from 0 to 255). Here's how the quantization process might look:
        1. Scale the Numbers: multiply each number by the maximum value representable by 8 bits (255). This step aligns the floating point range with the 8-bit integer range; [0.25 x 255, 0.5 x 255, 0.75 x 255, 1.0 x 255] = [63.75, 127.5, 191.25, 255]
        2. Round to Nearest Integer: [64, 128, 191, 255]
        


* Advantage: Reduces the memory footprint and computational cost even more than LoRA. This is especially beneficial when deploying models to environments with limited resources or when fine-tuning multiple models or tasks simultaneously.

## Let's Dive into Fine-Tuning Llama2


I've been recommended to try out Llama, Mistral, Yi, OLMo, etc. Here, let's try with Llama. Since this is my first time fine-tuning LLMs, I'm planning to document my entire journey in case any of you encounter similar struggles. 

First, I think I'll fine-tune the 7b-param model with QLoRA. As I need better hardware, I guess I'll use Colab (or Colab Pro if needed) and upload a separate file.